# Product Marketer with LangGraph and Responses API 🛍️📝✨

In [ ]:
!pip install langgraph-reflection langchain openevals openai

In [ ]:
from langgraph_reflection import create_reflection_graph
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START, END
from typing import TypedDict
from openevals.llm import create_llm_as_judge
from dotenv import load_dotenv

In [3]:
import requests
from io import BytesIO
from openai import OpenAI

client = OpenAI()

def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    return result.id

# Replace with your own file path or URL
file_id = create_file(client, "hiking_products.pdf")

file-Vjy1VBTgmnNbmDu6e9cPaT


In [4]:
vector_store = client.vector_stores.create(
    name="knowledge_base"
)
print(vector_store.id)

vs_67dd3eba12f48191ab289a29335d40b9


In [5]:
result = client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)

In [6]:
result = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)
print(result)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-Vjy1VBTgmnNbmDu6e9cPaT', created_at=1742552765, last_error=None, object='vector_store.file', status='completed', usage_bytes=32314, vector_store_id='vs_67dd3eba12f48191ab289a29335d40b9', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-Vjy1VBTgmnNbmDu6e9cPaT', last_id='file-Vjy1VBTgmnNbmDu6e9cPaT')


In [15]:
load_dotenv()

model = init_chat_model(model="openai:gpt-4o", use_responses_api=True)

openai_vector_store_ids = [
    vector_store.id,  # your IDs here
]

tools = [{"type": "web_search_preview"}, 
         {"type": "file_search",
        "vector_store_ids": openai_vector_store_ids}]

llm_with_tools = model.bind_tools(tools)

In [27]:
prompt="""
I'm on the marketing team for an ecommerce camping store called Contoso Outdoors.
Write me a short article that advertises the tents and sleeping bags we have in the hiking products file.
Make sure to name the specific products you include in the article. 
You should also look for the latest trends for camping in the summer in California. and include those
in the article. make sure to cite the sources you used. 
You should also include trending places to camp in California and link to information about those places.
The article should use a friendly and approachable tone.
"""

# article = llm_with_tools.invoke(f'{prompt}') 

In [17]:
# Define the main assistant model that will generate responses
def call_model(state):
    """Process the user query with a large language model."""

    print('Creating your article... 📝')
    
    return {"messages": llm_with_tools.invoke(state["messages"])}

# Define a basic graph for the main assistant
assistant_graph = (
    StateGraph(MessagesState)
    .add_node(call_model)
    .add_edge(START, "call_model")
    .add_edge("call_model", END)
    .compile()
)

In [33]:
from IPython.display import Markdown, display

# Define the tool that the judge can use to indicate the response is acceptable
class Finish(TypedDict):
    """Tool for the judge to indicate the response is acceptable."""

    finish: bool


# Define a more detailed critique prompt with specific evaluation criteria
critique_prompt = """You are an expert judge evaluating AI responses. Your task is to critique the AI assistant's latest response in the conversation below.

Evaluate the response based on these criteria:
1. Accuracy - Is the information correct and factual?
2. Completeness - Does it fully address the user's query?
3. Clarity - Is the explanation clear and well-structured?
4. Helpfulness - Does it provide actionable and useful information?
5. Safety - Does it avoid harmful or inappropriate content?

If the response meets ALL criteria satisfactorily, set pass to True.

If you find ANY issues with the response, do NOT set pass to True. Instead, provide specific and constructive feedback in the comment key and set pass to False.

Be detailed in your critique so the assistant can understand exactly how to improve.

<response>
{outputs}
</response>"""


# Define the judge function with a more robust evaluation approach
def judge_response(state, config):
    """Evaluate the assistant's response using a separate judge model."""
    evaluator = create_llm_as_judge(
        prompt=critique_prompt,
        model="openai:o3-mini",
        feedback_key="pass",
    )

    print('Evaluating the article... 🧐')
    eval_result = evaluator(outputs=state["messages"][-1].content, inputs=None)

    if eval_result["score"]:
        print("✅ Response approved by editor")
        print("")
        print("Here's your article:")
        
        # Render and display as Markdown
        display(Markdown(state["messages"][-1].text()))
        # print(state["messages"][-1].text())
        return
    else:
        # Otherwise, return the judge's critique as a new user message
        print("⚠️ Judge requested improvements")
        return {"messages": [{"role": "user", "content": eval_result["comment"]}]}


# Define the judge graph
judge_graph = (
    StateGraph(MessagesState)
    .add_node(judge_response)
    .add_edge(START, "judge_response")
    .add_edge("judge_response", END)
    .compile()
)


In [ ]:
# Create the complete reflection graph
reflection_app = create_reflection_graph(assistant_graph, judge_graph)
reflection_app = reflection_app.compile()



# Example query that might need improvement
example_query = [
    {
        "role": "user",
        "content": f"{prompt}",
    }
]

# Process the query through the reflection system
print("Running Content Generator and Editor with Reflection")
print("")
result = reflection_app.invoke({"messages": example_query})

Running Content Generator and Editor with Reflection

Creating your article... 📝
